In [716]:
import numpy as np
import pandas as pd 

df = pd.read_csv('IMDB Dataset.csv',sep = ',')

df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [717]:
#looking for any null values
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [718]:
import tensorflow as tf

In [719]:
print(tf.__version__)

2.15.0


In [720]:
df.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [721]:
df['sentiment_new'] = np.where(df['sentiment'] == 'positive',1,0)

df.drop('sentiment',axis = 1, inplace = True)

df['sentiment_new'].value_counts()

sentiment_new
1    25000
0    25000
Name: count, dtype: int64

In [838]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


In [ ]:
# Word cloud for all the Negative sentiment reviews

text= ""

def get_all_text(t):
    global text
    text = text+t+" "
    
df.review[df.sentiment_new == 1].apply(get_all_text)

wordcloud = WordCloud(
    width= 1200,
    height= 800,
    stopwords= ENGLISH_STOP_WORDS,
    background_color= 'white'
).generate(text)

plt.figure(figsize= (12,8))
plt.imshow(wordcloud, interpolation= 'bilinear')
plt.axis('off')

In [ ]:
# Word cloud for all the Negative sentiment reviews

text= ""

def get_all_text(t):
    global text
    text = text+t+" "
    
df.review[df.sentiment_new == 0].apply(get_all_text)

wordcloud = WordCloud(
    width= 1200,
    height= 800,
    stopwords= ENGLISH_STOP_WORDS,
    background_color= 'white'
).generate(text)

plt.figure(figsize= (12,8))
plt.imshow(wordcloud, interpolation= 'bilinear')
plt.axis('off')

In [724]:
#Getting independent variables
reviews = df.copy()

X = reviews.drop('sentiment_new',axis = 1)

#Getting dependent variables

y = reviews.drop('review',axis = 1)

print(X.head())
print(X.shape)

print() 

print(y.head())
print(y.shape)


                                              review
0  One of the other reviewers has mentioned that ...
1  A wonderful little production. <br /><br />The...
2  I thought this was a wonderful way to spend ti...
3  Basically there's a family where a little boy ...
4  Petter Mattei's "Love in the Time of Money" is...
(50000, 1)

   sentiment_new
0              1
1              1
2              1
3              0
4              1
(50000, 1)


In [725]:
reviews = X.copy()

reviews['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [726]:
import nltk 
import re
from nltk.corpus import stopwords

In [727]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sredekar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [728]:
reviews['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [729]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(reviews)):
    reviewed = re.sub('[^a-zA-Z]', ' ', reviews['review'][i])
    reviewed = reviewed.lower()
    reviewed = reviewed.split()
    
    reviewed = [ps.stem(word) for word in reviewed if not word in stopwords.words('english')] # removing stop words
    reviewed = ' '.join(reviewed)
    corpus.append(reviewed)

In [730]:
corpus[0]

'one review mention watch oz episod hook right exactli happen br br first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word br br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away br br would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch da

In [731]:
max_length_in_sentence = [len(sentences.split()) for sentences in corpus]

max_length_in_sentence.sort(reverse= True)
max_length_in_sentence

[1442,
 1182,
 1175,
 1174,
 955,
 940,
 936,
 887,
 859,
 844,
 842,
 799,
 747,
 740,
 737,
 701,
 701,
 696,
 666,
 661,
 656,
 646,
 643,
 636,
 634,
 629,
 620,
 620,
 618,
 615,
 610,
 610,
 605,
 600,
 599,
 598,
 598,
 595,
 595,
 594,
 594,
 593,
 593,
 591,
 591,
 591,
 591,
 589,
 587,
 587,
 586,
 586,
 584,
 584,
 583,
 582,
 582,
 581,
 581,
 580,
 579,
 578,
 578,
 578,
 577,
 577,
 577,
 576,
 576,
 575,
 574,
 574,
 574,
 574,
 574,
 573,
 573,
 571,
 570,
 569,
 569,
 569,
 569,
 568,
 568,
 567,
 567,
 567,
 566,
 564,
 564,
 564,
 563,
 563,
 563,
 563,
 562,
 561,
 560,
 560,
 560,
 560,
 559,
 559,
 559,
 559,
 558,
 558,
 557,
 557,
 557,
 557,
 557,
 556,
 556,
 556,
 556,
 556,
 556,
 555,
 554,
 554,
 554,
 554,
 553,
 552,
 551,
 551,
 551,
 551,
 550,
 550,
 550,
 550,
 549,
 549,
 549,
 549,
 549,
 549,
 548,
 548,
 547,
 547,
 547,
 547,
 547,
 547,
 547,
 546,
 546,
 545,
 545,
 544,
 544,
 543,
 543,
 542,
 542,
 541,
 541,
 541,
 541,
 540,
 540,
 540,


In [809]:
# Create a backup for the corpus 

#corpus_backup = corpus.copy()

corpus = corpus_backup.copy()

corpus_df = pd.DataFrame({'Sentences': corpus})

# Add a column with the lengths of sentences
corpus_df['Sentence Length'] = [len(sentence.split()) for sentence in corpus_df['Sentences']]

corpus_df = pd.concat([corpus_df,df['sentiment_new']],axis = 1)

# Display the DataFrame
print(corpus_df)



                                               Sentences  Sentence Length  \
0      one review mention watch oz episod hook right ...              168   
1      wonder littl product br br film techniqu unass...               92   
2      thought wonder way spend time hot summer weeke...               88   
3      basic famili littl boy jake think zombi closet...               70   
4      petter mattei love time money visual stun film...              133   
...                                                  ...              ...   
49995  thought movi right good job creativ origin fir...               81   
49996  bad plot bad dialogu bad act idiot direct anno...               61   
49997  cathol taught parochi elementari school nun ta...              117   
49998  go disagre previou comment side maltin one sec...              112   
49999  one expect star trek movi high art fan expect ...               68   

       sentiment_new  
0                  1  
1                  1  
2     

In [811]:
# How much will be size of data when we subset it based on lenght of sentences 

print(corpus_df[corpus_df['Sentence Length'] <= 150].shape)

(37712, 3)


In [812]:
# Adjusting the lengths so that modelling operation will not take more memory 
corpus_df = corpus_df[corpus_df['Sentence Length'] <= 150]

print(corpus_df.shape)


(37712, 3)


In [813]:
corpus = [i for i in corpus_df['Sentences']]

print(len(corpus))

print()

corpus

37712



['wonder littl product br br film techniqu unassum old time bbc fashion give comfort sometim discomfort sens realism entir piec br br actor extrem well chosen michael sheen got polari voic pat truli see seamless edit guid refer william diari entri well worth watch terrificli written perform piec master product one great master comedi life br br realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear play knowledg sens particularli scene concern orton halliwel set particularli flat halliwel mural decor everi surfac terribl well done',
 'thought wonder way spend time hot summer weekend sit air condit theater watch light heart comedi plot simplist dialogu witti charact likabl even well bread suspect serial killer may disappoint realiz match point risk addict thought proof woodi allen still fulli control style mani us grown love br br laugh one woodi comedi year dare say decad never impress scarlet johanson manag tone sexi imag jump right 

## OHE

In [814]:
### Vocabulary size

Total_words = [word for sentence in corpus for word in sentence.split()]

unique_words = set(Total_words)

count_unique_words = len(unique_words)

print(f'Put the vocabulary size  = {count_unique_words}')


Put the vocabulary size  = 50159


In [815]:
# Vocabulary size 

voc_size= count_unique_words


In [816]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[18938,
  15045,
  44425,
  8841,
  8841,
  8239,
  48414,
  46756,
  40389,
  32467,
  14200,
  39654,
  49306,
  19073,
  19266,
  11266,
  37086,
  7764,
  7625,
  46347,
  8841,
  8841,
  26698,
  25042,
  4479,
  19107,
  50086,
  1207,
  35273,
  23553,
  3914,
  37032,
  24832,
  802,
  3481,
  19199,
  15743,
  20783,
  42010,
  12709,
  1281,
  4479,
  49914,
  764,
  24610,
  33442,
  12423,
  46347,
  41317,
  44425,
  7860,
  27276,
  41317,
  39267,
  41419,
  8841,
  8841,
  7764,
  37257,
  4375,
  31783,
  15045,
  26708,
  43038,
  28573,
  1741,
  23588,
  3830,
  39140,
  48414,
  17315,
  26602,
  13874,
  25838,
  23026,
  37086,
  25485,
  39820,
  849,
  50134,
  35681,
  43738,
  25485,
  7317,
  35681,
  11668,
  625,
  8730,
  41624,
  15638,
  4479,
  44306],
 [11530,
  18938,
  38084,
  45976,
  32467,
  34658,
  14074,
  23354,
  47149,
  30452,
  29499,
  11802,
  764,
  30088,
  43072,
  39267,
  47885,
  36232,
  41873,
  22899,
  6376,
  3247,
  7689,


In [818]:
# Redefining dependent variables

y = corpus_df.drop(['Sentences','Sentence Length'],axis = 1)

y

,sentiment_new
1,1
2,1
3,0
4,1
5,1
...,...
49995,1
49996,0
49997,0
49998,0


### Embedding Representation

In [820]:
sent_length = 150 #max(len(i) for i in onehot_repr) 
print(sent_length)
embedded_docs = pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

150
[[18938 15045 44425 ...     0     0     0]
 [11530 18938 38084 ...     0     0     0]
 [36898 21469 15045 ...     0     0     0]
 ...
 [ 9347  9291  5514 ...     0     0     0]
 [20281 30041  7451 ...     0     0     0]
 [ 7860 26621 25467 ...     0     0     0]]


In [821]:
embedded_docs.shape

(37712, 150)

In [822]:
corpus[0]

'wonder littl product br br film techniqu unassum old time bbc fashion give comfort sometim discomfort sens realism entir piec br br actor extrem well chosen michael sheen got polari voic pat truli see seamless edit guid refer william diari entri well worth watch terrificli written perform piec master product one great master comedi life br br realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear play knowledg sens particularli scene concern orton halliwel set particularli flat halliwel mural decor everi surfac terribl well done'

In [823]:
onehot_repr[0]

[18938,
 15045,
 44425,
 8841,
 8841,
 8239,
 48414,
 46756,
 40389,
 32467,
 14200,
 39654,
 49306,
 19073,
 19266,
 11266,
 37086,
 7764,
 7625,
 46347,
 8841,
 8841,
 26698,
 25042,
 4479,
 19107,
 50086,
 1207,
 35273,
 23553,
 3914,
 37032,
 24832,
 802,
 3481,
 19199,
 15743,
 20783,
 42010,
 12709,
 1281,
 4479,
 49914,
 764,
 24610,
 33442,
 12423,
 46347,
 41317,
 44425,
 7860,
 27276,
 41317,
 39267,
 41419,
 8841,
 8841,
 7764,
 37257,
 4375,
 31783,
 15045,
 26708,
 43038,
 28573,
 1741,
 23588,
 3830,
 39140,
 48414,
 17315,
 26602,
 13874,
 25838,
 23026,
 37086,
 25485,
 39820,
 849,
 50134,
 35681,
 43738,
 25485,
 7317,
 35681,
 11668,
 625,
 8730,
 41624,
 15638,
 4479,
 44306]

## LSTM

In [824]:
## Creating model
embedding_vector_features=80 ##features representation ##Hyperparameters #every word will be represneted by 40 different features 
            
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) ##Embedding layer 
model.add(LSTM(100)) #NO of neurons # hyperparamter
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 150, 80)           4012720   
                                                                 
 lstm_19 (LSTM)              (None, 100)               72400     
                                                                 
 dense_19 (Dense)            (None, 1)                 101       
                                                                 
Total params: 4085221 (15.58 MB)
Trainable params: 4085221 (15.58 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [825]:
len(embedded_docs),y.shape

(37712, (37712, 1))

In [826]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [827]:
X_final.shape,y_final.shape

((37712, 150), (37712, 1))

In [828]:
X_final[0]

array([18938, 15045, 44425,  8841,  8841,  8239, 48414, 46756, 40389,
       32467, 14200, 39654, 49306, 19073, 19266, 11266, 37086,  7764,
        7625, 46347,  8841,  8841, 26698, 25042,  4479, 19107, 50086,
        1207, 35273, 23553,  3914, 37032, 24832,   802,  3481, 19199,
       15743, 20783, 42010, 12709,  1281,  4479, 49914,   764, 24610,
       33442, 12423, 46347, 41317, 44425,  7860, 27276, 41317, 39267,
       41419,  8841,  8841,  7764, 37257,  4375, 31783, 15045, 26708,
       43038, 28573,  1741, 23588,  3830, 39140, 48414, 17315, 26602,
       13874, 25838, 23026, 37086, 25485, 39820,   849, 50134, 35681,
       43738, 25485,  7317, 35681, 11668,   625,  8730, 41624, 15638,
        4479, 44306,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [829]:
corpus[0]

'wonder littl product br br film techniqu unassum old time bbc fashion give comfort sometim discomfort sens realism entir piec br br actor extrem well chosen michael sheen got polari voic pat truli see seamless edit guid refer william diari entri well worth watch terrificli written perform piec master product one great master comedi life br br realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear play knowledg sens particularli scene concern orton halliwel set particularli flat halliwel mural decor everi surfac terribl well done'

In [830]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [831]:
print(X_train.shape)
print()
print(X_test.shape)
print()
print(y_train.shape)
print()
print(y_test.shape)

(25267, 150)

(12445, 150)

(25267, 1)

(12445, 1)


### Model Training

In [832]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
395/395 [==============================] - 74s 182ms/step - loss: 0.6934 - accuracy: 0.5015 - val_loss: 0.6930 - val_accuracy: 0.5053
Epoch 2/10
395/395 [==============================] - 80s 201ms/step - loss: 0.6668 - accuracy: 0.6042 - val_loss: 0.6536 - val_accuracy: 0.6319
Epoch 3/10
395/395 [==============================] - 76s 193ms/step - loss: 0.6664 - accuracy: 0.6009 - val_loss: 0.6677 - val_accuracy: 0.5913
Epoch 4/10
395/395 [==============================] - 78s 197ms/step - loss: 0.6554 - accuracy: 0.6133 - val_loss: 0.6626 - val_accuracy: 0.6187
Epoch 5/10
395/395 [==============================] - 83s 210ms/step - loss: 0.6656 - accuracy: 0.6123 - val_loss: 0.6592 - val_accuracy: 0.6232
Epoch 6/10
395/395 [==============================] - 88s 221ms/step - loss: 0.6630 - accuracy: 0.6149 - val_loss: 0.6566 - val_accuracy: 0.6215
Epoch 7/10
395/395 [==============================] - 79s 199ms/step - loss: 0.6785 - accuracy: 0.5576 - val_loss: 0.6922 - val_ac

### Performance Metrics And Accuracy

In [833]:
y_pred=model.predict(X_test)

389/389 [==============================] - 27s 69ms/step


In [834]:
y_pred=np.where(y_pred > 0.5, 1,0) 

##if you do not want throshould of 0.5 use AUC ROC Curve

In [835]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[5615,  654],
       [ 833, 5343]], dtype=int64)

In [836]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.880514262756127

In [837]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.90      0.88      6269
           1       0.89      0.87      0.88      6176

    accuracy                           0.88     12445
   macro avg       0.88      0.88      0.88     12445
weighted avg       0.88      0.88      0.88     12445

